In [1]:
import os

from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, TimestampType, StringType, DoubleType, LongType
from pyspark.sql.functions import udf
from textblob import TextBlob

In [2]:
DATABASE_HOST = os.environ.get('DB_HOST', '')
DATABASE_PORT = os.environ.get('DB_PORT', '')
DATABASE_NAME = os.environ.get('DB_NAME', '')
DATABASE_USER = os.environ.get('DB_USER', '')
DATABASE_PASS = os.environ.get('DB_PASS', '')
ENV = os.environ.get('ENV', 'dev')
KAFKA_CONNECTION_STRING = os.environ.get('KAFKA_CONNECTION_STRING', 'kafka-broker:9092')
KAFKA_TOPIC_NAME = os.environ.get('KAFKA_TOPIC_NAME', 'us-election-tweet')

ENV = 'dev' if ENV == '' else ENV
KAFKA_CONNECTION_STRING = 'kafka-broker:9092' if ENV == 'dev' else KAFKA_CONNECTION_STRING

In [3]:
spark = (
    SparkSession.builder
    .appName('5003-project')
    .master('spark://spark-master:7077')
    .config("spark.files.overwrite", "true")
    .config('spark.driver.extraClassPath', '/opt/drivers/postgresql-42.2.19.jar')
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2')
    .enableHiveSupport()
    .getOrCreate()
)

:: loading settings :: url = jar:file:/opt/conda/envs/5003-project/lib/python3.9/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-517fb1ec-8398-450a-8887-a5695f6efd65;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.2 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.2 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 1178ms :: artifacts dl 19ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central in [default]

In [4]:
schema = (
    StructType()
    .add("created_at", TimestampType(), False)
    .add("tweet_id", DoubleType(), False)
    .add("tweet", StringType(), True)
    .add("likes", DoubleType(), True)
    .add("retweet_count", DoubleType(), True)
    .add("source", StringType(), True)
    .add("user_id", DoubleType(), True)
    .add("user_name", StringType(), True)
    .add("user_screen_name", StringType(), True)
    .add("user_description", StringType(), True)
    .add("user_join_date", TimestampType(), True)
    .add("user_followers_count", DoubleType(), True)
    .add("user_location", StringType(), True)
    .add("lat", DoubleType(), True)
    .add("long", DoubleType(), True)
    .add("city", StringType(), True)
    .add("country", StringType(), True)
    .add("continent", StringType(), True)
    .add("state", StringType(), True)
    .add("state_code", StringType(), True)
    .add("collected_at", TimestampType(), True)
)

### Streaming

In [5]:
df = (spark
    .readStream
    .format("kafka")
    .option("kafka.bootstrap.servers", KAFKA_CONNECTION_STRING)
    .option("subscribe", KAFKA_TOPIC_NAME)
    .load()
    .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")
)


In [5]:
query = df.writeStream \
            .format("console") \
            .option("truncate","false") \
            .start() \
            .awaitTermination()

21/11/12 08:51:13 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-61f69238-84d4-48f0-85c7-179cf758dfa3. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+---+-----+
|key|value|
+---+-----+
+---+-----+



ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/opt/conda/envs/5003-project/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3444, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_86/3284785342.py", line 1, in <module>
    query = df.writeStream \
  File "/opt/conda/envs/5003-project/lib/python3.9/site-packages/pyspark/sql/streaming.py", line 101, in awaitTermination
    return self._jsq.awaitTermination()
  File "/opt/conda/envs/5003-project/lib/python3.9/site-packages/py4j/java_gateway.py", line 1303, in __call__
    answer = self.gateway_client.send_command(command)
  File "/opt/conda/envs/5003-project/lib/python3.9/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/conda/envs/5003-project/lib/python3.9/site-packages/py4j/java_gateway.py", line 1200, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/opt/conda/envs/5003-project

TypeError: object of type 'NoneType' has no len()

In [14]:
df.start()
df.awaitTermination() 

AttributeError: 'DataFrame' object has no attribute 'print'

### Streaming

In [ ]:
# def postgres_sink(df, batch_id):
#     url = f'jdbc:postgresql://{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}?user={DATABASE_USER}&password={DATABASE_PASS}'
#     df.write.jdbc(url, 'sentiment', mode='append')
    
# write_table = (df
#     .writeStream
#     .outputMode("append") 
#     .foreachBatch(postgres_sink)
#     .start()
#     .awaitTermination())

### Batching

In [ ]:
from pyspark.sql import DataFrameWriter
my_writer = DataFrameWriter(df)

url = f'jdbc:postgresql://{DATABASE_HOST}:{DATABASE_PORT}/{DATABASE_NAME}?user={DATABASE_USER}&password={DATABASE_PASS}'
my_writer.jdbc(url, 'sentiment', mode='append')